In [4]:
import pandas as pd
import numpy as np
import xlrd
from plotly.graph_objs import Scatter, layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
from urllib.request import urlopen
import json
import numpy as np
import re
cf.go_offline()###这两句是离线生成图片的设置
cf.set_config_file(offline=True, world_readable=True)
plotly.offline.init_notebook_mode(connected=True)

In [5]:
df = pd.read_excel('outer.xlsx')

In [6]:
ans = np.zeros(100)

In [9]:
per_bt = 0.02
per_gold = 0.01

In [17]:
g1, g2 = 0.28, 0.57
b1, b2 = 0.56, 0.69
length = df.shape[0]

In [ ]:
g1 = np.arange(0.28, 0.32, 0.01)
g2 = np.arange(0.56, 0.60, 0.01)
b1 = np.arange(0.55, 0.60, 0.01)
b2 = np.arange(0.69, 0.72, 0.01)

In [24]:
trade_g, trade_b = np.array([]), np.array([])

In [25]:
for i in range(1,df.shape[0]):
    if i > 1:
        trade_g = np.append(trade_g, df['黄金持有份额'][i - 1] - df['黄金持有份额'][i - 2])
        trade_b = np.append(trade_b, df['比特币持有份额'][i - 1] - df['比特币持有份额'][i - 2])
    bt = df['比特币买入评分'][i]
    df['当前黄金利润'][i]=df['当前黄金利润'][i-1]+df['黄金涨幅'][i-1]*df['黄金持有份额'][i-1]
    df['当前比特币利润'][i]=df['当前比特币利润'][i-1]+df['比特币涨幅'][i-1]*df['比特币持有份额'][i-1]
    df['黄金持有份额'][i]=df['黄金持有份额'][i-1]
    df['现金持有份额'][i]=df['现金持有份额'][i-1]
    df['比特币持有份额'][i]=df['比特币持有份额'][i-1]
    if df['DealDay'][i]:  # 判断交易日
        gold = df['黄金买入评分'][i]
        if gold > g2:
            if bt > b2:# 两者都可买入时
                if gold-g2>(bt-b2)*2:  # 买入黄金
                    df['黄金持有份额'][i]=df['黄金持有份额'][i-1]+df['现金持有份额'][i-1]*gold*(1-per_gold)/df['USD (PM)'][i-1]
                    df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*gold
#                     print('两者选黄金',df['黄金持有份额'][i],df['现金持有份额'][i],df['现金持有份额'][i-1],gold,1-per_gold,df['USD (PM)'][i-1])
                else:  # 买入比特币
                    df['比特币持有份额'][i]=df['比特币持有份额'][i-1]+df['现金持有份额'][i-1]*bt*(1-per_bt)/df['Value'][i-1]
                    df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*bt
#                     print('两者选比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
            else: # 只能买黄金时
                df['黄金持有份额'][i]=df['黄金持有份额'][i-1]+df['现金持有份额'][i-1]*gold*(1-per_gold)/df['USD (PM)'][i-1]
                df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*gold
#                 print('只能买黄金',df['黄金持有份额'][i])
        if gold<g1:  # 买入黄金
            df['黄金持有份额'][i]=df['黄金持有份额'][i-1]-df['黄金持有份额'][i-1]*(1+gold-g2)
            df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['黄金持有份额'][i-1]*(1+gold-g2)*(1-per_gold)*df['USD (PM)'][i-1]
#             print('卖黄金',df['黄金持有份额'][i],df['现金持有份额'][i])
        continue    
    if bt > b2:  # 买入比特币
        df['比特币持有份额'][i]=df['比特币持有份额'][i-1]+df['现金持有份额'][i-1]*bt*2*(1-per_bt)/df['Value'][i-1]
        df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*bt
#         print('买比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
    if bt < b1:  # 卖出比特币
        df['比特币持有份额'][i]=df['比特币持有份额'][i-1]-df['比特币持有份额'][i-1]*(1+gold-b1)
        df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['比特币持有份额'][i-1]*(1+gold-b1)*(1-per_gold)*df['Value'][i-1]
#         print('卖比特币',df['比特币持有份额'][i],df['现金持有份额'][i])

In [67]:
total_Profit = np.zeros((4, length))

In [68]:
p = 0
for per_gold in np.arange(0.01, 0.016, 0.005):
    for per_bt in np.arange(0.01, 0.016, 0.005): 
        for i in range(1,df.shape[0]):
                # if i > 1:
                #     trade_g = np.append(trade_g, df['黄金持有份额'][i - 1] - df['黄金持有份额'][i - 2])
                #     trade_b = np.append(trade_b, df['比特币持有份额'][i - 1] - df['比特币持有份额'][i - 2])
                bt = df['比特币买入评分'][i]
                df['当前黄金利润'][i]=df['当前黄金利润'][i-1]+df['黄金涨幅'][i-1]*df['黄金持有份额'][i-1]
                df['当前比特币利润'][i]=df['当前比特币利润'][i-1]+df['比特币涨幅'][i-1]*df['比特币持有份额'][i-1]
                df['黄金持有份额'][i]=df['黄金持有份额'][i-1]
                df['现金持有份额'][i]=df['现金持有份额'][i-1]
                df['比特币持有份额'][i]=df['比特币持有份额'][i-1]
                if df['DealDay'][i]:  # 判断交易日
                    gold = df['黄金买入评分'][i]
                    if gold > g2:
                        if bt > b2:# 两者都可买入时
                            if gold-g2>(bt-b2)*2:  # 买入黄金
                                df['黄金持有份额'][i]=df['黄金持有份额'][i-1]+df['现金持有份额'][i-1]*gold*(1-per_gold)/df['USD (PM)'][i-1]
                                df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*gold
            #                     print('两者选黄金',df['黄金持有份额'][i],df['现金持有份额'][i],df['现金持有份额'][i-1],gold,1-per_gold,df['USD (PM)'][i-1])
                            else:  # 买入比特币
                                df['比特币持有份额'][i]=df['比特币持有份额'][i-1]+df['现金持有份额'][i-1]*bt*(1-per_bt)/df['Value'][i-1]
                                df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*bt
            #                     print('两者选比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
                        else: # 只能买黄金时
                            df['黄金持有份额'][i]=df['黄金持有份额'][i-1]+df['现金持有份额'][i-1]*gold*(1-per_gold)/df['USD (PM)'][i-1]
                            df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*gold
            #                 print('只能买黄金',df['黄金持有份额'][i])
                    if gold<g1:  # 买入黄金
                        df['黄金持有份额'][i]=df['黄金持有份额'][i-1]-df['黄金持有份额'][i-1]*(1+gold-g2)
                        df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['黄金持有份额'][i-1]*(1+gold-g2)*(1-per_gold)*df['USD (PM)'][i-1]
            #             print('卖黄金',df['黄金持有份额'][i],df['现金持有份额'][i])
                    continue    
                if bt > b2:  # 买入比特币
                    df['比特币持有份额'][i]=df['比特币持有份额'][i-1]+df['现金持有份额'][i-1]*bt*2*(1-per_bt)/df['Value'][i-1]
                    df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*bt
            #         print('买比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
                if bt < b1:  # 卖出比特币
                    df['比特币持有份额'][i]=df['比特币持有份额'][i-1]-df['比特币持有份额'][i-1]*(1+gold-b1)
                    df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['比特币持有份额'][i-1]*(1+gold-b1)*(1-per_gold)*df['Value'][i-1]
            #         print('卖比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
        df['总资产']=df['黄金持有份额']*df['USD (PM)']+df['比特币持有份额']*df['Value']+df['现金持有份额']
        total_Profit[p] = df['总资产']
        p += 1

In [26]:
df['总资产']=df['黄金持有份额']*df['USD (PM)']+df['比特币持有份额']*df['Value']+df['现金持有份额']
df['黄金资产']=df['黄金持有份额']*df['USD (PM)']
df['比特币资产']=df['比特币持有份额']*df['Value']

In [27]:
df['总资产']

0       1.000000e+03
1       1.000000e+03
2       1.000000e+03
3       1.000000e+03
4       1.000000e+03
            ...     
1821    1.508344e+07
1822    1.527311e+07
1823    1.402344e+07
1824    1.386855e+07
1825    1.393132e+07
Name: 总资产, Length: 1826, dtype: float64

In [21]:
df['总资产'][length - 1]

13931321.374005452

In [54]:
z = total_Profit
x, y = np.arange(0.01, 0.051, 0.005), np.arange(0.01, 0.051, 0.005)
fig = go.Figure()
fig.add_trace(go.Surface(z=z, x=x, y=y))
fig.add_trace(go.Scatter3d(x=[0.02], y=[0.01], z=[total_Profit[0, 2]],
    mode="markers+text",
    marker={'size':4},
    text=['(0.02, 0.01)'],
    textfont={'color':["#bc80bd"], 'family':'Balto, sans-serif', 'size':[10]},
    textposition=['middle right']
))
fig.update_layout(title={'text':'Maximum revenue at different transaction costs', 'x':0.5, 'y':0.86}, autosize=False,
                  width=800, height=600,
                  scene=dict(xaxis_title='TC of Bitcoin', yaxis_title='TC of Gold',
                            zaxis_title='Revenue ($)', aspectratio=dict(x=2, y=2, z=1)),
                  margin=dict(l=55, r=50, b=65, t=90))
fig.show()

In [52]:
ans[0][1]

IndexError: invalid index to scalar variable.

In [22]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['总资产'],
    mode='lines',
    name='Asset Distribution',
))
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['黄金资产'],
    mode='lines',
    name='Total gold assets',
))
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['比特币资产'],
    mode='lines',
    name='Total Bitcoin Assets',
))
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['现金持有份额'],
    mode='lines',
    name='Total Cash',
))
fig.update_layout(title={'text':'Asset Distribution', 'x':0.5, 'y':0.85}, xaxis_title='Date',
    yaxis_title='U.S. dollars($)', width=800, plot_bgcolor='rgb(230, 245, 231)' )
fig.show()

In [38]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

fig = make_subplots(rows=1, cols=2, subplot_titles=("Gold Trading Volume", "Bitcoin Trading Volume"))
fig.add_trace(go.Scatter(
        x=df['Date'][1:],
        y=trade_g,
        mode='markers',
        name='Gold trading volume',
        marker=dict(size=np.abs(trade_g) / 60, color=trade_g, colorscale='Sunset')
        
    ),
    row=1, col=1
)
fig.add_trace(go.Scatter(
        x=df['Date'][1:],
        y=trade_b,
        mode='markers',
        name='Bitcoin transaction volume',
        marker=dict(size=np.abs(trade_b) / 8, color=trade_b, colorscale='Viridis')
    ),
    row=1, col=2
)
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Troy Ounce", row=1, col=1)
fig.update_yaxes(title_text="Bitcoin", row=1, col=2)
fig.update_layout(title={'text':'Trading strategy', 'x':0.5, 'y':0.88}, width=900, plot_bgcolor='rgb(230, 245, 231)',showlegend = False )
fig.show()

In [57]:
X = np.zeros((9, 8))

In [55]:
ans = total_Profit

In [58]:
for i in range(0, 9):
    for j in range(1, 9):
        X[i][j - 1] = (ans[i][j] - ans[i][j - 1]) / ans[i][j - 1]

In [59]:
X

array([[-0.04598826, -0.0461877 , -0.04638901, -0.04659222, -0.04679736,
        -0.04700444, -0.04721351, -0.04742458],
       [-0.04598387, -0.04618327, -0.04638454, -0.04658771, -0.0467928 ,
        -0.04699984, -0.04720886, -0.04741988],
       [-0.04597965, -0.04617901, -0.04638023, -0.04658336, -0.0467884 ,
        -0.0469954 , -0.04720438, -0.04741536],
       [-0.0459756 , -0.04617492, -0.04637611, -0.04657919, -0.04678419,
        -0.04699115, -0.04720008, -0.04741102],
       [-0.04597174, -0.04617102, -0.04637217, -0.04657521, -0.04678018,
        -0.04698709, -0.04719599, -0.04740689],
       [-0.04596809, -0.04616733, -0.04636845, -0.04657145, -0.04677638,
        -0.04698326, -0.04719212, -0.04740298],
       [-0.04596467, -0.04616388, -0.04636496, -0.04656793, -0.04677282,
        -0.04697967, -0.04718848, -0.04739931],
       [-0.04596149, -0.04616067, -0.04636172, -0.04656466, -0.04676952,
        -0.04697633, -0.04718512, -0.04739591],
       [-0.04595858, -0.04615773

In [60]:
np.mean(X, axis=1)

array([-0.04669964, -0.0466951 , -0.04669072, -0.04668653, -0.04668254,
       -0.04667876, -0.04667521, -0.04667193, -0.04666892])

In [23]:
np.std(X, axis=1)

array([0.00034091, 0.00034044, 0.00033997, 0.00033949, 0.000339  ,
       0.00033851, 0.00033801, 0.00033751, 0.000337  ])

In [61]:
X = np.zeros((8, 9))

In [62]:
for j in range(0, 9):
    for i in range(1, 9):
        X[i - 1][j] = (ans[i][j] - ans[i - 1][j]) / ans[i - 1][j]

In [63]:
np.mean(X, axis=0)

array([-0.10927781, -0.10927435, -0.10927085, -0.10926731, -0.10926374,
       -0.10926014, -0.1092565 , -0.10925282, -0.1092491 ])

In [64]:
np.std(X, axis=0)

array([0.0015563 , 0.00155676, 0.00155722, 0.00155769, 0.00155816,
       0.00155863, 0.00155912, 0.0015596 , 0.00156009])

In [69]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=df['Date'][900:],
    y=total_Profit[0][900:],
    mode='lines',
    name='Origin Line',
))
fig.add_trace(go.Scatter(
    x=df['Date'][900:],
    y=total_Profit[1][900:],
    mode='lines',
    name='Change Line 1',
))
fig.add_trace(go.Scatter(
    x=df['Date'][900:],
    y=total_Profit[2][900:],
    mode='lines',
    name='Change Line 2',
))
fig.add_trace(go.Scatter(
    x=df['Date'][900:],
    y=total_Profit[3][900:],
    mode='lines',
    name='Change Line 3',
))
fig.update_layout(title={'text':'Trends in asset changes at different transaction costs', 'x':0.5, 'y':0.85}, xaxis_title='Date',
    yaxis_title='U.S. dollars($)', width=800, plot_bgcolor='rgb(230, 215, 211)' )
fig.show()